# Fine Tuning the gemma-270m model

Neste notebook vamos realizar uma demonstração de um simples fine tuning com o modelo gemma-270m.
## Motivação
Este modelo é extremamente pequeno e eficiente para o seu tamanho, o que reduz drasticamente os gastos computacionais para a realização de um fine tuning. Cogitei realizar o fine tuning dentro do ambiente da Bedrock, porém como todo o projeto principal já utilizou o Bedrock para diversas tarefas, optei por realizar o fine tuning de outra forma, a fim de mostrar um espectro maior de habilidades. Além disso, pessoalmente já queria tentar realizar o fine tuning desde modelo desde o seu lançamento, há alguns meses.  

In [ ]:
# Se estiver no Google Colab tira o comentario abaixo depois da primeira execução
!pip install unsloth datasets trl

In [ ]:
import torch
import pandas as pd
from datasets import load_dataset
from unsloth import FastModel
import json

In [ ]:
max_seq_len = 2048

In [ ]:
# https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Gemma3_(270M).ipynb#scrollTo=-Xbb0cuLzwgf
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-270m-it",
    max_seq_length = max_seq_len,
    load_in_4bit = False,
    load_in_8bit = False,
    full_finetuning = False,
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Para realizar o fine tuning, é necessário primeiramente transformar os dados do dataset utilizado em um formato em que o modelo possa aprender algo, cada modelo possui um template para uso e por isso vamos importar o template específico do gemma-3.

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma3",
)

Cada exemplo deve manter um papdrão de exemplo de conversa, a função abaixo realizará esta tarefa e será aplicada via map().

In [ ]:
def processar_chatml(exemplo):
    context = exemplo['context']
    prompt = f"Context: {context}\nQuestion: {exemplo['question']}"
    completion = f"Answer: {exemplo['answers']['text']}"
    return {"conversations": [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": completion}
    ]}

As conversas padronizadas são então aplicadas utilizando a função apply_chat_template.

In [ ]:
def formatar_prompts(exemplos):
   convos = exemplos["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }


O dataset utilizado, 'Stanford Question Answering Dataset (SQuAD)', possui diversas perguntas e resposta, sendo utilizado neste fine tuning para melhorar a capacidade do modelo em responder perguntas trivia. Naturalmente, pode-se utilizar diversos tipos de dataset para melhorar as capacidades do modelo em um tópico específico. Testei diversos datasets antes de encontrar este, mas devido a complexidade dos datasets e a proposta do desafio de aplicar um fine tunin simples, optei por utilizar este mais generalista para otimizar o uso do tempo em outras tarefas.

As funções apresentadas são aplicadas via map() nos dados do dataset e então separadas em dataset de treinamento e dataset de teste.

In [ ]:

dataset = load_dataset("rajpurkar/squad")
dataset = dataset.map(processar_chatml)
dataset = dataset.map(formatar_prompts, batched = True)
print(len(dataset['train']))
dataset_misturado = dataset['train'].shuffle(seed=42)
train_dataset = dataset_misturado.select(range(70000))
test_dataset = dataset_misturado.select(range(70000, len(dataset['train'])))

print(f"Training set size: {len(train_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Antes de iniciar o treinamento, vamos verificar uma resposta do modelo:

In [ ]:
messages = [
    {'role': 'system','content':dataset['conversations'][10][0]['content']},
    {"role" : 'user', 'content' : dataset['conversations'][10][1]['content']}
]

print(test_dataset['conversations'][10][0]['content'])
print(test_dataset['conversations'][10][1]['content'])
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
).removeprefix('<bos>')

print("Model answer:")
from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 250,
    temperature = 1, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

pre_model = model

Agora é possível iniciar o processo de treinamento, onde colocamos o dataset de treinamento e o de verificação dos resultados, bem como alguns hiperparâmertos importantes para o treinamento. Para o leitor, os hiperparâmetros que mais modifiquei foram: 'warmup_steps', 'max_steps', 'num_train_epochs', 'weight_decay' e 'lr_scheduler_type'.


In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_dataset, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        #num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 5e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir="outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ").replace("<end_of_turn>", "")

Agora podemos iniciar o treinamento, nos meus testes o loss caiu de ~3.3 a ~1.8, mostrando que o modelo realmente aprendeu alguma coisa.

In [ ]:
trainer_stats = trainer.train()

Podemos então obter uma resposta do modelo com fine tuning:

In [ ]:
print("Modelo com fine tuning:")
indice_do_item = 12 # Troque para obter novas perguntas e respostas
messages = [
    {'role': 'system','content':test_dataset['conversations'][indice_do_item][0]['content']},
    {"role" : 'user', 'content' : test_dataset['conversations'][indice_do_item][1]['content']}
]
print(test_dataset['conversations'][indice_do_item][0]['content'])
print(test_dataset['conversations'][indice_do_item][1]['content'])
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
).removeprefix('<bos>')
print("Model answer:")
from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors="pt").to("cuda"),
    max_new_tokens=250,
    temperature=1, top_p=0.95, top_k=64,
    streamer=TextStreamer(tokenizer, skip_prompt=True)
)

E o modelo sem fine tuning.

In [ ]:
print("Modelo sem fine tuning:")
indice_do_item = 12 # Troque para obter novas perguntas e respostas
messages = [
    {'role': 'system','content':test_dataset['conversations'][indice_do_item][0]['content']},
    {"role" : 'user', 'content' : test_dataset['conversations'][indice_do_item][1]['content']}
]
print(test_dataset['conversations'][indice_do_item][0]['content'])
print(test_dataset['conversations'][indice_do_item][1]['content'])
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
).removeprefix('<bos>')
print("Model answer:")
from transformers import TextStreamer
_ = pre_model.generate(
    **tokenizer(text, return_tensors="pt").to("cuda"),
    max_new_tokens=250,
    temperature=1, top_p=0.95, top_k=64,
    streamer=TextStreamer(tokenizer, skip_prompt=True)
)

Como pode ser visto, o modelo com fine tuning apresenta

In [ ]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")